In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("/scratch/network/cs9095/cs551/eQTL_data/eQTL_joint.tsv.gz", sep="\t")
genes = pd.read_csv("/scratch/network/cs9095/cs551/GWAS_data/ENSEMBL_genes.tsv").ENSEMBL.to_list()

In [ ]:
blist, plist, slist = [], [], []
for i in range(15):
    blist.append(f"beta_{i}")
    plist.append(f"pvalue_{i}")  
    slist.append(f"SE_{i}")
    
tlist = []
for i in range(15):
    tlist.append([f"beta_{i}", f"SE_{i}", f"pvalue_{i}"])

In [ ]:
# SNP filtering:

goi = []
signs = []
pvals = [0.1, 0.05, 0.01]

for p in pvals:
    g_temp = []
    s_temp = []

    for gene in genes:
        sub_df = df.loc[(df.gene_id == gene)]
        for beta, pval in zip(blist, plist):
            
            # 1 == positive, 2 == negative
            sub_df.loc[:,beta] = [1 if i else 2 for i in sub_df[beta].ge(0).tolist()] 
            # only take p-values less than 1e-3
            sub_df.loc[:,pval] = [int(i) for i in sub_df[pval].le(p).tolist()] 
            
        sub_df = sub_df.drop(columns=slist)
        sub_df = sub_df.reset_index()
    
        # baesd on pval element-wise erase non-signif.
        con = pd.DataFrame(sub_df[plist].to_numpy() * sub_df[blist].to_numpy())  
        con[["pos", "neg"]] = 0
        con.pos = con.isin([1]).sum(axis=1).tolist()
        con.neg = con.isin([2]).sum(axis=1).tolist()
        
        # take only if 3+ tissues agree in the same direction
        if np.sum(con.pos >= 3) > np.sum(con.neg >= 3): 
            ind = con[(con.pos >= 3)].index
            sign = 1
        else:
            ind = con[(con.neg >= 3)].index
            sign = -1
        if not sub_df.iloc[ind].empty:
            g_temp.append(gene)
            s_temp.append(sign)
    goi.append(g_temp)
    signs.append(s_temp)    

In [ ]:
# Does the confidence threshold alter the number of genes selected?
[len(i) for i in goi] 
# ^ [16, 16, 15]: Only in the last threshold drop

# Are the 16 genes in the 0.1 set and 0.05 set the same?
goi[0] == goi[1] 
# ^ True: the genes in 0.1 and 0.05 are identical

# Are the genes in the 0.01 set all in the 0.05 set?
[gene in goi[1] for gene in goi[2]] 
# ^ [True...True]: the genes in 0.01 are all in 0.05

# Which gene was droppped?
[gene for gene in goi[1] if (not gene in goi[2])] 
# ^ ENSG00000073792: insulin like growth factor 2 mRNA binding protein 2

In [ ]:
# Setup confidence threshold analysis dataframe
total = pd.DataFrame()
for i, j, p in zip(goi, signs, pvals):
    oi = pd.DataFrame()
    oi["gene"] = [item for item in i]
    oi[f"sign"] = [1 if item == "pos" else -1 for item in j]
    oi["pval"] = str(p)
    total = pd.concat([total, oi])    

In [ ]:
# Does the confidence threshold alter the consenus sign of signifcant betas? 

fig, axes = plt.subplots(int(len(total.gene.unique())/4),4, figsize=(15,15), dpi=120, 
                         sharey=True, sharex=True)
axes = np.ravel(axes)

for gene, ax in zip(total.gene.unique(), axes):
    sns.pointplot(data=total[total.gene == gene], x="pval", y="sign", hue="gene", 
                  palette="pastel", ax=ax)
plt.savefig("/scratch/network/cs9095/cs551/figures/sign_changes.png")

---

In [ ]:
# Plots of eQTL data for signficant Genes: 

# abs. beta , and neg. log of p_value
# large abs betas roughly correlated with signifcance
# however, discrepancies can be attributed ot differences in the mag of the est. SE
# this is accoutned for in our model, whihc is a weighted mix-effects model.
# this emphasies taking into accoutn estimatation uncertaintiyh in our model.

fig, axes = plt.subplots(len(goi),2,figsize=(15,50), dpi=100)
axes = np.ravel(axes)
    
for gene, a, b in zip(goi, axes[::2], axes[1::2]):
    at = pd.read_csv(f"/scratch/network/cs9095/cs551/eQTL_data/eQTL_genes/eQTL_{gene}.tsv.gz", sep="\t")
    at = at.drop(columns=["gene_id"])
    at = at.set_index(["variant_id"])
    sns.heatmap(data=at[blist].T.apply(np.absolute), ax=a, cmap="RdPu")
    sns.heatmap(data=at[plist].T.apply(np.log).apply(lambda x: x * -1), ax=b, cmap="RdPu")
    a.title.set_text(f'{gene} Betas')
    b.title.set_text(f'{gene} P-values')

plt.tight_layout()
plt.savefig(f"/scratch/network/cs9095/cs551/figures/sum_{gene}_beta_pval.png")